In [95]:
pip install pandas sqlalchemy mysql-connector-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import mysql.connector



In [ ]:
# ---------------------- CONFIGURACIÓN DE CONEXIÓN ----------------------

DB_CONFIG = {
    'user': 'root',
    'password': 'Burningcaravan123',
    'host': 'localhost',
    'port': '3306',
    'database': 'ab_test_lab'
}

def crear_engine():
    connection_string = f"mysql+mysqlconnector://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
    return create_engine(connection_string)

def verificar_conexion():
    try:
        engine = crear_engine()
        with engine.connect() as conn:
            print("✅ Conexión exitosa a MySQL")
    except Exception as e:
        print("❌ No se pudo conectar a MySQL")
        print(e)



In [ ]:
# ---------------------- CARGA DE CLIENT_PROFILES ----------------------

def cargar_client_profiles(path_txt):
    df_demo = pd.read_csv(path_txt, sep=',')
    print(df_demo.head())
    print(df_demo.info())
    print(df_demo.describe(include='all'))

    # Borrar datos anteriores (mantener la tabla)
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()
    cursor.execute("DELETE FROM client_profiles;")
    conn.commit()
    cursor.close()
    conn.close()

    # Cargar nuevos datos
    engine = crear_engine()
    df_demo.to_sql('client_profiles', con=engine, if_exists='append', index=False)
    print("✅ Tabla 'client_profiles' cargada correctamente.")



In [ ]:
# ---------------------- CARGA DE DIGITAL_FOOTPRINTS ----------------------

def cargar_digital_footprints(path1, path2):
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path2)

    for df in [df1, df2]:
        df['date_time'] = pd.to_datetime(df['date_time'], errors='coerce')
        print(df[df['date_time'].isnull()])
        df.dropna(subset=['date_time'], inplace=True)
        df.reset_index(drop=True, inplace=True)

    print(f"df1: {len(df1)} filas")
    print(f"df2: {len(df2)} filas")

    df_web = pd.concat([df1, df2], ignore_index=True)

    # Validación de columnas
    expected_columns = {'client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time'}
    if set(df_web.columns) != expected_columns:
        raise ValueError(f"Columnas incorrectas. Esperadas: {expected_columns}, encontradas: {set(df_web.columns)}")

    # Tipos de datos
    for col in ['client_id', 'visitor_id', 'visit_id', 'process_step']:
        df_web[col] = df_web[col].astype(str)
    df_web['date_time'] = pd.to_datetime(df_web['date_time'], errors='coerce')

    if df_web['date_time'].isnull().any():
        raise ValueError("❌ Hay valores no parseables en 'date_time'")

    # Carga a MySQL
    try:
        engine = crear_engine()
        with engine.begin() as connection:
            df_web.to_sql(
                'digital_footprints',
                con=connection,
                if_exists='replace',
                index=False,
                chunksize=10000
            )
        print("✅ Tabla 'digital_footprints' cargada correctamente.")
    except SQLAlchemyError as e:
        print("❌ Error al cargar datos:", e)



In [ ]:
# ---------------------- CARGA DE EXPERIMENT_ROSTER ----------------------

def cargar_experiment_roster(path_experiment):
    df = pd.read_csv(path_experiment)

    # Validaciones
    expected_cols = {'client_id', 'Variation'}
    if set(df.columns) != expected_cols:
        raise ValueError(f"Columnas incorrectas. Esperadas: {expected_cols}, encontradas: {set(df.columns)}")

    df['Variation'] = df['Variation'].str.strip().str.lower()
    valores_validos = {'control', 'test'}
    if not set(df['Variation'].dropna()).issubset(valores_validos):
        raise ValueError(f"Valores inválidos en 'Variation': {set(df['Variation']) - valores_validos}")

    if df['client_id'].duplicated().any():
        raise ValueError("❌ Hay IDs duplicados en 'client_id'.")

    df.columns = ['client_id', 'variation']

    # Carga
    try:
        engine = crear_engine()
        with engine.begin() as connection:
            df.to_sql(
                'experiment_roster',
                con=connection,
                if_exists='replace',
                index=False
            )
        print("✅ Tabla 'experiment_roster' cargada correctamente.")
    except SQLAlchemyError as e:
        print("❌ Error al cargar datos:", e)



In [ ]:
# ---------------------- MAIN ----------------------

if __name__ == '__main__':
    verificar_conexion()

    # Rutas
    ruta_demo = 'C:/Users/juana/OneDrive/Escritorio/ironhack/ab_testing_project/data/df_final_demo.txt'
    ruta_web1 = 'C:/Users/juana/OneDrive/Escritorio/ironhack/ab_testing_project/data/df_final_web_data_pt_1.txt'
    ruta_web2 = 'C:/Users/juana/OneDrive/Escritorio/ironhack/ab_testing_project/data/df_final_web_data_pt_2.txt'
    ruta_exp = 'C:/Users/juana/OneDrive/Escritorio/ironhack/ab_testing_project/data/df_final_experiment_clients.txt'

    cargar_client_profiles(ruta_demo)
    cargar_digital_footprints(ruta_web1, ruta_web2)
    cargar_experiment_roster(ruta_exp)

from sqlalchemy import create_engine
import pandas as pd

# Configuración de conexión
db_user = 'root'
db_password = 'Burningcaravan123'
db_host = 'localhost'
db_port = '3306'
db_name = 'ab_test_lab'

# Crear engine
connection_string = f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)

# Lista de tablas que quieres verificar
tablas = ['client_profiles', 'digital_footprints', 'experiment_roster']

# Mostrar primeras filas
for tabla in tablas:
    print(f"\n--- Primeras filas de '{tabla}' ---")
    try:
        df = pd.read_sql(f"SELECT * FROM {tabla} LIMIT 5;", con=engine)
        print(df)
    except Exception as e:
        print(f"❌ Error al leer la tabla '{tabla}':", e)
